# 복지시설 데이터 동별 매칭

⇒ 돌봄시설 API

규호: 공원, 도서관, 체육시설, 대중교통

하은: 경찰범죄통계, 5대 범죄 구별 범죄율, 주차, 57번

재영: 초/ 중/고등/대학교, 도로, (각 초등학교가 커버하는 범위?), 56번, 31번

동명: 상권, 학원가, 공장, 병원, 69번, 58번

소연: 건폐율, 전용면적, 각 행정동 안에 주택개수(코드)

⇒ 행정동별로 개수, 평균

⇒ X, Y 좌표 (위도, 경도)

⇒ 동별로 없으면? 구별로도

⇒ 개수, 면적(규모)

In [78]:
# 필요한 라이브러리 임포트&설치
!pip install dbfread
!pip install haversine
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
import pickle
import re
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'

서울시 전체에 분포한

공원, 도서관, 대중교통(지하철/버스), 체육시설, +미술관박물관

을 동별로 몇개씩 있는지 분배한 데이터프레임을 만들면 되는데,

동별이 안되면 구별로 하면 됨.

위도경도 따져주고

가능하면 부가정보도(넓이라던가 수용인원이라던가 연식이라던가)

1. 일단 주소/위도경도만 남기고 다 없애자(흔적은 columns해서 남길것)

2. re findall로 동만 추출한다거나, 구 추출해서 매칭해주기(새로운 열)

3. 데이터 적은거로 구별 중심으로 해서 가까운곳 매칭하는것(지하철/버스) 해서 옆에 ~동 이렇게 하던지 동옆에 ~개 이렇게 하던지.


In [79]:
###필요한 데이터 불러오기
#대중교통-버스정류소
bus = pd.read_excel(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시버스정류소좌표데이터(2020.03.06).xlsx', encoding='cp949')
#공원
park = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시 주요 공원현황.csv', encoding='cp949')
#박물관
museum = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울특별시_박물관미술관정보_20180821.csv', encoding='cp949')
#도서관
lib = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\전국도서관표준데이터.csv', encoding='cp949')

###없는데이터 목록
#체육시설-하은이 기다려야함
# sports = pd.read_csv()
#대중교통-지하철-못찾았었음(그나마1-4호선)

###위치매핑등을 위해 동별 구분단위 불러오기
#동사무소
office = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\전국공공시설개방정보표준데이터.csv', encoding='cp949')
#구 좌표
gu_loc = pd.read_csv(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시 행정구역 시군구 정보 (좌표계_ WGS1984).csv', encoding='cp949')
#행정동목록
dong = pd.read_excel(r'C:\Users\이규호\NLP\github\seoul_contest\data\서울시_행정동_법정동_20200814.xlsx', encoding='cp949')


출처:

버스:https://data.seoul.go.kr/dataList/OA-15067/S/1/datasetView.do

공원: http://data.seoul.go.kr/dataList/OA-394/S/1/datasetView.do;jsessionid=C03761A8C8B90B16BD4AD7EAD8E06367.new_portal-svr-21

미술/박물관:http://data.seoul.go.kr/dataList/OA-15272/F/1/datasetView.do

도서관:https://www.data.go.kr/data/15013109/standard.do

지하철:http://data.seoul.go.kr/dataList/OA-12035/A/1/datasetView.do;jsessionid=E738D56520A14676B34670384E77126E.new_portal-svr-11

동사무소:https://www.data.go.kr/data/15013117/standard.do

구좌표:https://data.seoul.go.kr/dataList/OA-11677/S/1/datasetView.do

행정동목록: 소연이

# 원본 데이터 오픈

In [80]:
bus

,표준ID,ARS-ID,정류장명,X좌표,Y좌표,비고
0,100000001,1001,종로2가사거리,126.987750,37.569765,NaN
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,NaN
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,NaN
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,NaN
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,NaN
...,...,...,...,...,...,...
11275,124000334,25995,우성아파트,127.139339,37.550386,NaN
11276,124000333,25996,우성아파트,127.140046,37.550643,NaN
11277,124000332,25997,조일약국,127.123596,37.533630,NaN
11278,124000331,25998,성내시장,127.125497,37.536155,NaN


In [81]:
park

,공원번호,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107000,4.503956e+05,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,3,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120000,4.488527e+05,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do
2,4,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267000,4.371640e+05,127.019847,37.426449,http://grandpark.seoul.go.kr/
3,5,서울숲,당초 골프장 승마장 등이 있던 뚝섬일대를 주거업무 지역으로 개발할 경우 약 4조원에...,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,http://parks.seoul.go.kr/template/common/img/p...,NaN,지하철 분당선 서울숲역 3번출구 도보로 약5분 2호선 뚝섬역 8번출구 도보로 약 1...,http://parks.seoul.go.kr/file/info/view.do?fId...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),서울숲 관리사무소,02-460-2905,203695.432000,4.492907e+05,127.041798,37.543072,http://parks.seoul.go.kr/template/sub/seoulfor...
4,6,월드컵공원,월드컵공원은 서울의 서쪽에 위치하여 1978년부터 1993년까지 15년간 서울시민이...,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,http://parks.seoul.go.kr/template/common/img/p...,지하철 6호선 월드컵경기장역 하차 → 1번 출구로 나온 후 직진 → 큰길(도로)이 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),월드컵공원,02-300-5500,190658.070000,4.515988e+05,126.878907,37.571805,http://parks.seoul.go.kr/template/sub/worldcup...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,1378,용마도시자연공원(사가정공원),2005년 4월 13일 개장한 사가정공원은 면목동 산 50번지 일대의 면목약수터지구...,109635㎡,2005.4.13,피크닉장 놀이터 체력단련장 체육관,NaN,NaN,NaN,NaN,http://parks.seoul.go.kr/file/info/view.do?fId...,중랑구,서울특별시 중랑구 면목7동 산50,중랑구청 공원녹지과,02-2094-2993,NaN,NaN,127.095997,37.579762,NaN
128,1379,문화비축기지,매봉산자락에 위치한 문화비축기지는 폐산업시설인 '마포석유비축기지'를 재생하여 역사적...,140022㎡,2017.9.1,6개의탱크와 문화마당 녹지,NaN,http://parks.seoul.go.kr/upload/seditorMulti/2...,NaN,? 일반사항 - 공원은 연중무휴이나 전시관은 월요일 휴관입니다. - 공원을 ...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 증산로 87,문화비축기지,02-376-8410,114264.460400,3.712093e+06,126.893245,37.571718,http://parks.seoul.go.kr/template/sub/culturet...
129,1380,경춘선숲길,일제 강점기때 우리 민족의 자본으로 만든 최초의 철도시설로 1939년 7월 25일 ...,211392㎡,NaN,조경시설 : 텃밭 잔디광장 철길 산책로 등 건축시설 : 무궁화호 객차(방문자센터) ...,NaN,NaN,지하철 이용시 지하철 7호선 하계역 4번출구 미성/장미아파트 도로 450m직진 지하...,○ 6.3km (시 작 점 : 육사삼거리 중간지점 : 화랑대사거리 → 화랑대...,http://parks.seoul.go.kr/file/info/view.do?fId...,NaN,노원구 공릉2동 산 82-2,중부공원녹지사업소,02-2289-4012,127.076482,3.762708e+01,127.076482,37.627077,NaN
130,1381,율현공원,? 율현공원은 보금자리주택 세곡2지구 사업의 결과물로 만들어졌다. 공원은 주택지구의...,157535㎡,2016.11.1,관리시설 : 사무실 조경시설 : 정자(4개) 조형쉼터(10개) 조형파고라(3개) 수...,수목 : 소나무 이팝나무 등 110종 143069주 초화 : 맥문동 벌개미취 등...,http://parks.seoul.go.kr/upload/seditorMulti/2...,NaN,공원 인근에 서울공항 활주로가 위치하고 있어 공원 내에서 드론을 날리거나 헬륨가스 ...,http://parks.seoul.go.kr/file/info/view.do?fId...,강남구,서울특별시 강남구 율현동 밤고개로24길(율현동 56),동부공원녹지사업소,02-459-9

In [82]:
museum

,시설명,박물관미술관구분,소재지도로명주소,소재지지번주소,위도,경도,운영기관전화번호,운영기관명,운영홈페이지,편의시설정보,...,휴관정보,어른관람료,청소년관람료,어린이관람료,관람료기타정보,박물관미술관소개,교통안내정보,관리기관전화번호,관리기관명,데이터기준일자
0,둘리뮤지엄,공립,서울특별시 도봉구 시루봉로 1길 6,NaN,37.652432,127.027640,02-990-2200,도봉구,doolymuseum.or.kr,NaN,...,홈페이지참고,5000,5000,5000,NaN,NaN,NaN,02-990-2200,도봉구,2017-12-31
1,서대문자연사박물관,공립,서울특별시 서대문구 연희로 32길 51,NaN,37.576753,126.937858,02-330-8899,서대문구,http://namu.sdm.go.kr,NaN,...,홈페이지참고,6000,3000,2000,NaN,NaN,NaN,02-330-8899,서대문구,2017-12-31
2,서대문형무소역사관,공립,서울특별시 서대문구 통일로251,NaN,37.572442,126.959541,02-360-8590,서대문구,www.sscmc.or.kr,NaN,...,홈페이지참고,3000,1500,1000,NaN,NaN,NaN,02-360-8590,서대문구,2017-12-31
3,서울교육박물관,공립,서울특별시 종로구 북촌로 5길 48,NaN,37.581031,126.983694,02-2011-5780,서울특별시특별시교육청,http://edumuseum.sen.go.kr,NaN,...,홈페이지참고,0,0,0,NaN,NaN,NaN,02-2011-5780,서울특별시특별시교육청,2017-12-31
4,서울상상나라,공립,서울특별시 광진구 능동로 216,NaN,37.549458,127.086261,02-6450-9500,서울특별시특별시,www.seoulchildrensmuseum.org,NaN,...,홈페이지참고,4000,4000,4000,NaN,NaN,NaN,02-6450-9500,서울특별시특별시,2017-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,포스코미술관,사립,서울특별시 강남구 대치4동 892 포스코센터 내,NaN,37.505831,127.056088,02-3457-1665,㈜포스코,http://www.poscoartmuseum.org/,NaN,...,홈페이지참고,NaN,NaN,NaN,NaN,NaN,NaN,02-3457-1665,㈜포스코,2017-12-31
155,학아재미술관,사립,서울특별시 종로구 돈화문로 98,NaN,NaN,NaN,02-766-7647,김로이,www.hagajae.com,NaN,...,홈페이지참고,NaN,NaN,NaN,NaN,NaN,NaN,02-766-7647,김로이,2017-12-31
156,한미사진미술관,사립,"서울특별시 송파구 위례성대로 14 한미타워 19, 20층",NaN,37.516608,127.113970,02-418-1315,가현문화재단,www.photomuseum.or.kr,NaN,...,홈페이지참고,NaN,NaN,NaN,NaN,NaN,NaN,02-418-1315,가현문화재단,2017-12-31
157,헬로우뮤지움,사립,서울특별시 성동구 금호로 72,NaN,37.507358,127.040312,02-3217-4222,김효정\n(김이삭),www.hellomuseum.com,NaN,...,홈페이지참고,NaN,NaN,NaN,NaN,NaN,NaN,02-3217-4222,김효정\n(김이삭),2017-12-31


In [83]:
lib

,도서관명,시도명,시군구명,도서관유형,휴관일,평일운영시작시각,평일운영종료시각,토요일운영시작시각,토요일운영종료시각,공휴일운영시작시각,...,운영기관명,도서관전화번호,부지면적,건물면적,홈페이지주소,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,덕산아내프리미엄아파트 작은도서관,경상남도,거제시,작은도서관,"매주월요일, 공휴일",16:00,22:00,16:00,22:00,16:00,...,경상남도 거제시 덕산아내프리미엄1차아파트입주자대표회,055-681-9766,31675.0,142.00,NaN,34.866433,128.684936,2020-01-30,5370000,경상남도 거제시
1,강하작은도서관,경기도,양평군,작은도서관,"매주일,월/공휴일",09:00,18:00,09:00,18:00,00:00,...,경기도 양평군청,031-771-4302,634.0,179.00,http://www.yplib.go.kr,37.496177,127.411217,2020-03-06,4170000,경기도 양평군
2,강상작은도서관,경기도,양평군,작은도서관,"매주일,월/공휴일",09:00,18:00,09:00,18:00,00:00,...,경기도 양평군청,031-775-7483,3927.0,140.00,http://www.yplib.go.kr,37.480695,127.478067,2020-03-06,4170000,경기도 양평군
3,양동도서관,경기도,양평군,공공도서관,"신정,설,추석연휴",08:00,22:00,08:00,22:00,08:00,...,경기도 양평군청,031-770-2571,3434.0,1314.00,http://www.yplib.go.kr,37.419191,127.754688,2020-03-06,4170000,경기도 양평군
4,지평도서관,경기도,양평군,공공도서관,"신정,설,추석연휴",08:00,22:00,08:00,22:00,08:00,...,경기도 양평군청,031-770-2561,2215.0,1045.00,http://www.yplib.go.kr,37.473719,127.638517,2020-03-06,4170000,경기도 양평군
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,늘푸른 도서관 하동군,경상남도,하동군,작은도서관,매주월요일,11:00,17:00,13:00,17:00,13:00,...,경상남도 하동군청,032-123-4567,330.0,475.20,http://hadong.libekas4.net,34.123457,123.123457,2017-07-26,5440000,경상남도 하동군
3314,밀양시립영어도서관,경상남도,밀양시,공공도서관,"매주 월요일, 신정, 구정, 추석 연휴일, 기타 도서관장이 정한 날",09:00,18:00,09:00,18:00,09:00,...,경상남도 밀양시,055-359-6045,5047.0,1104.00,http://eng.myclib.or.kr,35.486850,128.752564,2017-03-31,5360000,경상남도 밀양시
3315,밀양시립도서관,경상남도,밀양시,공공도서관,"매주 월요일, 신정, 구정, 추석 연휴일, 기타 도서관장이 정한 날",08:00,23:00,08:00,23:00,08:00,...,경상남도 밀양시,055-359-6026,5629.0,5907.00,http://www.myclib.or.kr,35.486850,128.752564,2017-03-31,5360000,경상남도 밀양시
3316,나주시립도서관,전라남도,나주시,공공도서관,"매월 첫째, 셋째 월요일, 법정공휴일",08:00,23:00,08:00,23:00,08:00,...,나주시청,061-339-4581,2188.0,1215.00,www.najulib.go.kr,34.998874,126.709512,2017-04-03,4830000,전라남도 나주시


In [84]:
# sports

In [85]:
office

,개방시설명,개방장소명,개방시설유형구분,휴관일,평일운영시작시각,평일운영종료시각,주말운영시작시각,주말운영종료시각,유료사용여부,사용기준시간,...,소재지지번주소,관리기관명,담당부서명,사용안내전화번호,홈페이지주소,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,아르코공연연습센터 광주세미나실,소촌아트팩토리,강의실,월,10:00,18:00,10:00,18:00,N,NaN,...,광주광역시 광산구 소촌동 845,광산구청,문화예술과,062-960-3688,https://www.arko.or.kr/infra/,35.152627,126.791234,2020-07-20,3630000,광주광역시 광산구
1,아르코공연연습센터 광주리딩룸,소촌아트팩토리,강의실,월,10:00,18:00,10:00,18:00,N,NaN,...,광주광역시 광산구 소촌동 845,광산구청,문화예술과,062-960-3688,https://www.arko.or.kr/infra/,35.152627,126.791234,2020-07-20,3630000,광주광역시 광산구
2,송광종합사회복지관 소강당,송광종합사회복지관,강당,주말,09:00,18:00,00:00,00:00,Y,1,...,광주광역시 광산구 우산동 1603-1,광산구청,공동체복지과,062-941-8248,www.gwangsan.go.kr,35.159824,126.804145,2020-07-20,3630000,광주광역시 광산구
3,송광종합사회복지관 한사랑방,송광종합사회복지관,강의실,주말,09:00,18:00,00:00,00:00,Y,1,...,광주광역시 광산구 우산동 1603-1,광산구청,공동체복지과,062-941-8248,www.gwangsan.go.kr,35.159824,126.804145,2020-07-20,3630000,광주광역시 광산구
4,송정1동 행정복지센터 다목적실(2층),송정1동 행정복지센터,다목적실,주말,09:00,17:00,00:00,00:00,N,NaN,...,광주광역시 광산구 송정동 756-1,광산구청,송정1동 행정복지센터,062-960-7942,www.gwangsan.go.kr,35.140746,126.798783,2020-07-20,3630000,광주광역시 광산구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6717,여수문화홀,여수문화홀,강당,"행사없는 토,일요일,공휴일",08:00,22:00,08:00,22:00,Y,4,...,NaN,전라남도 여수시청,문화예술과,061-659-4692,NaN,34.760520,127.662136,2019-06-17,4810000,전라남도 여수시
6718,해오름전시실,해오름전시실,전시실,월요일,09:00,18:00,09:00,18:00,N,0,...,NaN,전라남도 여수시청,문화예술과,061-666-3766,NaN,34.739025,127.748151,2019-06-17,4810000,전라남도 여수시
6719,상주문화회관,상주문화회관,기타,명절,09:00,22:00,09:00,22:00,Y,13,...,NaN,경상북도 상주시청,문화예술과,054-537-6215,http://www.sangju.go.kr/culture/index.jsp,36.411915,128.159163,2019-05-27,5110000,경상북도 상주시
6720,청소년수련관,청소년수련관,풋살경기장,"명절(신정,구정, 추석)",09:00,21:00,09:00,18:00,Y,1,...,NaN,경상북도 상주시청,평생학습원,054-537-5562,http://youth.sangju.go.kr,36.429813,128.162789,2019-05-27,5110000,경상북도 상주시


In [86]:
gu_loc

,순번,시군구코드,시군구명_한글,시군구명_영문,ESRI_PK,위도,경도
0,1,11320,도봉구,Dobong-gu,0,37.665861,127.031767
1,2,11380,은평구,Eunpyeong-gu,1,37.617612,126.922700
2,3,11230,동대문구,Dongdaemun-gu,2,37.583801,127.050700
3,4,11590,동작구,Dongjak-gu,3,37.496504,126.944307
4,5,11545,금천구,Geumcheon-gu,4,37.460097,126.900155
5,6,11530,구로구,Guro-gu,5,37.495486,126.858121
6,7,11110,종로구,Jongno-gu,6,37.599100,126.986149
7,8,11305,강북구,Gangbuk-gu,7,37.646995,127.014716
8,9,11260,중랑구,Jungnang-gu,8,37.595379,127.093967
9,10,11680,강남구,Gangnam-gu,9,37.495985,127.066409


In [87]:
dong

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN
...,...,...,...,...,...,...,...,...
21717,5013061000,제주특별자치도,서귀포시,중문동,5013011400,대포동,20060701,NaN
21718,5013061000,제주특별자치도,서귀포시,중문동,5013011800,하원동,20060701,NaN
21719,5013062000,제주특별자치도,서귀포시,예래동,5013011900,색달동,20060701,NaN
21720,5013062000,제주특별자치도,서귀포시,예래동,5013012000,상예동,20060701,NaN


# 각 데이터 컬럼 보기

In [88]:
bus.columns

Index(['표준ID', 'ARS-ID', '정류장명', 'X좌표', 'Y좌표', '비고'], dtype='object')

In [89]:
park.columns

Index(['공원번호', '공원명', '공원개요', '면적', '개원일', '주요시설', '주요식물', '안내도', '오시는길',
       '이용시참고사항', '이미지', '지역', '공원주소', '관리부서', '전화번호', 'X좌표(GRS80TM)',
       'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기'],
      dtype='object')

In [90]:
museum.columns

Index(['시설명', '박물관미술관구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '운영기관전화번호',
       '운영기관명', '운영홈페이지', '편의시설정보', '평일관람시작시각', '평일관람종료시각', '공휴일관람시작시각',
       '공휴일관람종료시각', '휴관정보', '어른관람료', '청소년관람료', '어린이관람료', '관람료기타정보', '박물관미술관소개',
       '교통안내정보', '관리기관전화번호', '관리기관명', '데이터기준일자'],
      dtype='object')

In [91]:
lib.columns

Index(['도서관명', '시도명', '시군구명', '도서관유형', '휴관일', '평일운영시작시각', '평일운영종료시각',
       '토요일운영시작시각', '토요일운영종료시각', '공휴일운영시작시각', '공휴일운영종료시각', '열람좌석수', '자료수(도서)',
       '자료수(연속간행물)', '자료수(비도서)', '대출가능권수', '대출가능일수', '소재지도로명주소', '운영기관명',
       '도서관전화번호', '부지면적', '건물면적', '홈페이지주소', '위도', '경도', '데이터기준일자', '제공기관코드',
       '제공기관명'],
      dtype='object')

In [92]:
office.columns

Index(['개방시설명', '개방장소명', '개방시설유형구분', '휴관일', '평일운영시작시각', '평일운영종료시각', '주말운영시작시각',
       '주말운영종료시각', '유료사용여부', '사용기준시간', '사용료', '초과사용단위시간', '초과사용료', '수용가능인원수',
       '면적', '부대시설정보', '신청방법구분', '시설사진정보', '소재지도로명주소', '소재지지번주소', '관리기관명',
       '담당부서명', '사용안내전화번호', '홈페이지주소', '위도', '경도', '데이터기준일자', '제공기관코드',
       '제공기관명'],
      dtype='object')

In [93]:
gu_loc.columns

Index(['순번', '시군구코드', '시군구명_한글', '시군구명_영문', 'ESRI_PK', '위도', '경도'], dtype='object')

In [94]:
dong.columns

Index(['행정동코드', '시도명', '시군구명', '읍면동명', '법정동코드', '동리명', '생성일자', '말소일자'], dtype='object')

# 필요한 열만 남기는 전처리

우선, 시설의 이름과 주소, 위도경도가 필요하다. 연식/넓이 등은 나중에 컬럼보고 돌아와서 추가할 것

완성되면 xx_df로 이름짓기

In [95]:
bus

,표준ID,ARS-ID,정류장명,X좌표,Y좌표,비고
0,100000001,1001,종로2가사거리,126.987750,37.569765,NaN
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,NaN
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,NaN
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,NaN
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,NaN
...,...,...,...,...,...,...
11275,124000334,25995,우성아파트,127.139339,37.550386,NaN
11276,124000333,25996,우성아파트,127.140046,37.550643,NaN
11277,124000332,25997,조일약국,127.123596,37.533630,NaN
11278,124000331,25998,성내시장,127.125497,37.536155,NaN


In [97]:
bus_df = bus[['표준ID','ARS-ID','정류장명', 'X좌표', 'Y좌표']]
bus_df

,표준ID,ARS-ID,정류장명,X좌표,Y좌표
0,100000001,1001,종로2가사거리,126.987750,37.569765
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671
3,100000004,1004,종로2가.삼일교,126.987613,37.568579
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...,...,...
11275,124000334,25995,우성아파트,127.139339,37.550386
11276,124000333,25996,우성아파트,127.140046,37.550643
11277,124000332,25997,조일약국,127.123596,37.533630
11278,124000331,25998,성내시장,127.125497,37.536155


In [98]:
park

,공원번호,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107000,4.503956e+05,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,3,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120000,4.488527e+05,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do
2,4,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267000,4.371640e+05,127.019847,37.426449,http://grandpark.seoul.go.kr/
3,5,서울숲,당초 골프장 승마장 등이 있던 뚝섬일대를 주거업무 지역으로 개발할 경우 약 4조원에...,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,http://parks.seoul.go.kr/template/common/img/p...,NaN,지하철 분당선 서울숲역 3번출구 도보로 약5분 2호선 뚝섬역 8번출구 도보로 약 1...,http://parks.seoul.go.kr/file/info/view.do?fId...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),서울숲 관리사무소,02-460-2905,203695.432000,4.492907e+05,127.041798,37.543072,http://parks.seoul.go.kr/template/sub/seoulfor...
4,6,월드컵공원,월드컵공원은 서울의 서쪽에 위치하여 1978년부터 1993년까지 15년간 서울시민이...,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,http://parks.seoul.go.kr/template/common/img/p...,지하철 6호선 월드컵경기장역 하차 → 1번 출구로 나온 후 직진 → 큰길(도로)이 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),월드컵공원,02-300-5500,190658.070000,4.515988e+05,126.878907,37.571805,http://parks.seoul.go.kr/template/sub/worldcup...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,1378,용마도시자연공원(사가정공원),2005년 4월 13일 개장한 사가정공원은 면목동 산 50번지 일대의 면목약수터지구...,109635㎡,2005.4.13,피크닉장 놀이터 체력단련장 체육관,NaN,NaN,NaN,NaN,http://parks.seoul.go.kr/file/info/view.do?fId...,중랑구,서울특별시 중랑구 면목7동 산50,중랑구청 공원녹지과,02-2094-2993,NaN,NaN,127.095997,37.579762,NaN
128,1379,문화비축기지,매봉산자락에 위치한 문화비축기지는 폐산업시설인 '마포석유비축기지'를 재생하여 역사적...,140022㎡,2017.9.1,6개의탱크와 문화마당 녹지,NaN,http://parks.seoul.go.kr/upload/seditorMulti/2...,NaN,? 일반사항 - 공원은 연중무휴이나 전시관은 월요일 휴관입니다. - 공원을 ...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 증산로 87,문화비축기지,02-376-8410,114264.460400,3.712093e+06,126.893245,37.571718,http://parks.seoul.go.kr/template/sub/culturet...
129,1380,경춘선숲길,일제 강점기때 우리 민족의 자본으로 만든 최초의 철도시설로 1939년 7월 25일 ...,211392㎡,NaN,조경시설 : 텃밭 잔디광장 철길 산책로 등 건축시설 : 무궁화호 객차(방문자센터) ...,NaN,NaN,지하철 이용시 지하철 7호선 하계역 4번출구 미성/장미아파트 도로 450m직진 지하...,○ 6.3km (시 작 점 : 육사삼거리 중간지점 : 화랑대사거리 → 화랑대...,http://parks.seoul.go.kr/file/info/view.do?fId...,NaN,노원구 공릉2동 산 82-2,중부공원녹지사업소,02-2289-4012,127.076482,3.762708e+01,127.076482,37.627077,NaN
130,1381,율현공원,? 율현공원은 보금자리주택 세곡2지구 사업의 결과물로 만들어졌다. 공원은 주택지구의...,157535㎡,2016.11.1,관리시설 : 사무실 조경시설 : 정자(4개) 조형쉼터(10개) 조형파고라(3개) 수...,수목 : 소나무 이팝나무 등 110종 143069주 초화 : 맥문동 벌개미취 등...,http://parks.seoul.go.kr/upload/seditorMulti/2...,NaN,공원 인근에 서울공항 활주로가 위치하고 있어 공원 내에서 드론을 날리거나 헬륨가스 ...,http://parks.seoul.go.kr/file/info/view.do?fId...,강남구,서울특별시 강남구 율현동 밤고개로24길(율현동 56),동부공원녹지사업소,02-459-9

In [99]:
park_df = park[['공원명','지역','공원주소','X좌표(WGS84)','Y좌표(WGS84)']]
park_df

,공원명,지역,공원주소,X좌표(WGS84),Y좌표(WGS84)
0,남산도시자연공원,중구,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,과천시,경기도 과천시 대공원광장로 102,127.019847,37.426449
3,서울숲,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,월드컵공원,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805
...,...,...,...,...,...
127,용마도시자연공원(사가정공원),중랑구,서울특별시 중랑구 면목7동 산50,127.095997,37.579762
128,문화비축기지,마포구,서울특별시 마포구 증산로 87,126.893245,37.571718
129,경춘선숲길,NaN,노원구 공릉2동 산 82-2,127.076482,37.627077
130,율현공원,강남구,서울특별시 강남구 율현동 밤고개로24길(율현동 56),127.115594,37.472332


In [106]:
museum

,시설명,박물관미술관구분,소재지도로명주소,소재지지번주소,위도,경도,운영기관전화번호,운영기관명,운영홈페이지,편의시설정보,...,휴관정보,어른관람료,청소년관람료,어린이관람료,관람료기타정보,박물관미술관소개,교통안내정보,관리기관전화번호,관리기관명,데이터기준일자
0,둘리뮤지엄,공립,서울특별시 도봉구 시루봉로 1길 6,NaN,37.652432,127.027640,02-990-2200,도봉구,doolymuseum.or.kr,NaN,...,홈페이지참고,5000,5000,5000,NaN,NaN,NaN,02-990-2200,도봉구,2017-12-31
1,서대문자연사박물관,공립,서울특별시 서대문구 연희로 32길 51,NaN,37.576753,126.937858,02-330-8899,서대문구,http://namu.sdm.go.kr,NaN,...,홈페이지참고,6000,3000,2000,NaN,NaN,NaN,02-330-8899,서대문구,2017-12-31
2,서대문형무소역사관,공립,서울특별시 서대문구 통일로251,NaN,37.572442,126.959541,02-360-8590,서대문구,www.sscmc.or.kr,NaN,...,홈페이지참고,3000,1500,1000,NaN,NaN,NaN,02-360-8590,서대문구,2017-12-31
3,서울교육박물관,공립,서울특별시 종로구 북촌로 5길 48,NaN,37.581031,126.983694,02-2011-5780,서울특별시특별시교육청,http://edumuseum.sen.go.kr,NaN,...,홈페이지참고,0,0,0,NaN,NaN,NaN,02-2011-5780,서울특별시특별시교육청,2017-12-31
4,서울상상나라,공립,서울특별시 광진구 능동로 216,NaN,37.549458,127.086261,02-6450-9500,서울특별시특별시,www.seoulchildrensmuseum.org,NaN,...,홈페이지참고,4000,4000,4000,NaN,NaN,NaN,02-6450-9500,서울특별시특별시,2017-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,포스코미술관,사립,서울특별시 강남구 대치4동 892 포스코센터 내,NaN,37.505831,127.056088,02-3457-1665,㈜포스코,http://www.poscoartmuseum.org/,NaN,...,홈페이지참고,NaN,NaN,NaN,NaN,NaN,NaN,02-3457-1665,㈜포스코,2017-12-31
155,학아재미술관,사립,서울특별시 종로구 돈화문로 98,NaN,NaN,NaN,02-766-7647,김로이,www.hagajae.com,NaN,...,홈페이지참고,NaN,NaN,NaN,NaN,NaN,NaN,02-766-7647,김로이,2017-12-31
156,한미사진미술관,사립,"서울특별시 송파구 위례성대로 14 한미타워 19, 20층",NaN,37.516608,127.113970,02-418-1315,가현문화재단,www.photomuseum.or.kr,NaN,...,홈페이지참고,NaN,NaN,NaN,NaN,NaN,NaN,02-418-1315,가현문화재단,2017-12-31
157,헬로우뮤지움,사립,서울특별시 성동구 금호로 72,NaN,37.507358,127.040312,02-3217-4222,김효정\n(김이삭),www.hellomuseum.com,NaN,...,홈페이지참고,NaN,NaN,NaN,NaN,NaN,NaN,02-3217-4222,김효정\n(김이삭),2017-12-31


In [107]:
museum_df = museum[['시설명','소재지도로명주소','소재지지번주소','위도','경도','운영기관명','관리기관명']]
museum_df

,시설명,소재지도로명주소,소재지지번주소,위도,경도,운영기관명,관리기관명
0,둘리뮤지엄,서울특별시 도봉구 시루봉로 1길 6,NaN,37.652432,127.027640,도봉구,도봉구
1,서대문자연사박물관,서울특별시 서대문구 연희로 32길 51,NaN,37.576753,126.937858,서대문구,서대문구
2,서대문형무소역사관,서울특별시 서대문구 통일로251,NaN,37.572442,126.959541,서대문구,서대문구
3,서울교육박물관,서울특별시 종로구 북촌로 5길 48,NaN,37.581031,126.983694,서울특별시특별시교육청,서울특별시특별시교육청
4,서울상상나라,서울특별시 광진구 능동로 216,NaN,37.549458,127.086261,서울특별시특별시,서울특별시특별시
...,...,...,...,...,...,...,...
154,포스코미술관,서울특별시 강남구 대치4동 892 포스코센터 내,NaN,37.505831,127.056088,㈜포스코,㈜포스코
155,학아재미술관,서울특별시 종로구 돈화문로 98,NaN,NaN,NaN,김로이,김로이
156,한미사진미술관,"서울특별시 송파구 위례성대로 14 한미타워 19, 20층",NaN,37.516608,127.113970,가현문화재단,가현문화재단
157,헬로우뮤지움,서울특별시 성동구 금호로 72,NaN,37.507358,127.040312,김효정\n(김이삭),김효정\n(김이삭)


# 동사무소(행정복지센터)

In [15]:
office

,개방시설명,개방장소명,개방시설유형구분,휴관일,평일운영시작시각,평일운영종료시각,주말운영시작시각,주말운영종료시각,유료사용여부,사용기준시간,...,소재지지번주소,관리기관명,담당부서명,사용안내전화번호,홈페이지주소,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,아르코공연연습센터 광주세미나실,소촌아트팩토리,강의실,월,10:00,18:00,10:00,18:00,N,NaN,...,광주광역시 광산구 소촌동 845,광산구청,문화예술과,062-960-3688,https://www.arko.or.kr/infra/,35.152627,126.791234,2020-07-20,3630000,광주광역시 광산구
1,아르코공연연습센터 광주리딩룸,소촌아트팩토리,강의실,월,10:00,18:00,10:00,18:00,N,NaN,...,광주광역시 광산구 소촌동 845,광산구청,문화예술과,062-960-3688,https://www.arko.or.kr/infra/,35.152627,126.791234,2020-07-20,3630000,광주광역시 광산구
2,송광종합사회복지관 소강당,송광종합사회복지관,강당,주말,09:00,18:00,00:00,00:00,Y,1,...,광주광역시 광산구 우산동 1603-1,광산구청,공동체복지과,062-941-8248,www.gwangsan.go.kr,35.159824,126.804145,2020-07-20,3630000,광주광역시 광산구
3,송광종합사회복지관 한사랑방,송광종합사회복지관,강의실,주말,09:00,18:00,00:00,00:00,Y,1,...,광주광역시 광산구 우산동 1603-1,광산구청,공동체복지과,062-941-8248,www.gwangsan.go.kr,35.159824,126.804145,2020-07-20,3630000,광주광역시 광산구
4,송정1동 행정복지센터 다목적실(2층),송정1동 행정복지센터,다목적실,주말,09:00,17:00,00:00,00:00,N,NaN,...,광주광역시 광산구 송정동 756-1,광산구청,송정1동 행정복지센터,062-960-7942,www.gwangsan.go.kr,35.140746,126.798783,2020-07-20,3630000,광주광역시 광산구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6717,여수문화홀,여수문화홀,강당,"행사없는 토,일요일,공휴일",08:00,22:00,08:00,22:00,Y,4,...,NaN,전라남도 여수시청,문화예술과,061-659-4692,NaN,34.760520,127.662136,2019-06-17,4810000,전라남도 여수시
6718,해오름전시실,해오름전시실,전시실,월요일,09:00,18:00,09:00,18:00,N,0,...,NaN,전라남도 여수시청,문화예술과,061-666-3766,NaN,34.739025,127.748151,2019-06-17,4810000,전라남도 여수시
6719,상주문화회관,상주문화회관,기타,명절,09:00,22:00,09:00,22:00,Y,13,...,NaN,경상북도 상주시청,문화예술과,054-537-6215,http://www.sangju.go.kr/culture/index.jsp,36.411915,128.159163,2019-05-27,5110000,경상북도 상주시
6720,청소년수련관,청소년수련관,풋살경기장,"명절(신정,구정, 추석)",09:00,21:00,09:00,18:00,Y,1,...,NaN,경상북도 상주시청,평생학습원,054-537-5562,http://youth.sangju.go.kr,36.429813,128.162789,2019-05-27,5110000,경상북도 상주시


In [16]:
office.columns

Index(['개방시설명', '개방장소명', '개방시설유형구분', '휴관일', '평일운영시작시각', '평일운영종료시각', '주말운영시작시각',
       '주말운영종료시각', '유료사용여부', '사용기준시간', '사용료', '초과사용단위시간', '초과사용료', '수용가능인원수',
       '면적', '부대시설정보', '신청방법구분', '시설사진정보', '소재지도로명주소', '소재지지번주소', '관리기관명',
       '담당부서명', '사용안내전화번호', '홈페이지주소', '위도', '경도', '데이터기준일자', '제공기관코드',
       '제공기관명'],
      dtype='object')

In [33]:
office_need = office[['개방장소명','소재지도로명주소','소재지지번주소','위도','경도','제공기관명']]
office_need

,개방장소명,소재지도로명주소,소재지지번주소,위도,경도,제공기관명
0,소촌아트팩토리,광주광역시 광산구 소촌로85번길 14-9,광주광역시 광산구 소촌동 845,35.152627,126.791234,광주광역시 광산구
1,소촌아트팩토리,광주광역시 광산구 소촌로85번길 14-9,광주광역시 광산구 소촌동 845,35.152627,126.791234,광주광역시 광산구
2,송광종합사회복지관,광주광역시 광산구 우산로 17,광주광역시 광산구 우산동 1603-1,35.159824,126.804145,광주광역시 광산구
3,송광종합사회복지관,광주광역시 광산구 우산로 17,광주광역시 광산구 우산동 1603-1,35.159824,126.804145,광주광역시 광산구
4,송정1동 행정복지센터,광주광역시 광산구 광산로 70,광주광역시 광산구 송정동 756-1,35.140746,126.798783,광주광역시 광산구
...,...,...,...,...,...,...
6717,여수문화홀,전라남도 여수시 시청로 1,NaN,34.760520,127.662136,전라남도 여수시
6718,해오름전시실,"전라남도 여수시 자산공원길 54, 해상관제센터 1층 (종화동)",NaN,34.739025,127.748151,전라남도 여수시
6719,상주문화회관,경상북도 상주시 상산로 235,NaN,36.411915,128.159163,경상북도 상주시
6720,청소년수련관,경상북도 상주시 북상주로 24-42,NaN,36.429813,128.162789,경상북도 상주시


In [34]:
office_need.제공기관명.unique()

array(['광주광역시 광산구', '경기도 수원시', '경상남도 창원시', '충청북도 청주시', '충청북도 옥천군',
       '충청북도 제천시', '경기도 안산시', '전라북도 군산시', '강원도 인제군', '울산광역시도시공사',
       '경상북도 예천군', '대구광역시 달서구', '거제해양관광개발공사', '부산광역시 남구', '인천광역시 옹진군',
       '경상북도 포항시', '경상남도 고성군', '전라북도 정읍시', '대구광역시 서구', '경상북도 성주군',
       '충청남도 금산군', '경기도 광명시', '경상남도 거제시', '울산광역시 동구', '울산광역시 남구',
       '경상북도 봉화군', '전라남도 보성군', '부산광역시 수영구', '서울특별시 도봉구', '울산광역시남구도시관리공단',
       '전라북도 남원시', '경기도 하남시', '대구광역시 중구', '부산지방공단스포원', '대구광역시',
       '경상북도 울릉군', '강원도 양구군', '전라북도', '경기도 성남시', '전라북도 고창군', '서울특별시 종로구',
       '전라북도 장수군', '서울특별시 동대문구', '부산광역시 동래구', '인천광역시 계양구', '대구광역시 동구',
       '경상남도 산청군', '전라남도 무안군', '전라북도 임실군', '대구광역시교육청 대구광역시교육연수원',
       '서울특별시 구로구', '충청남도 홍성군', '울산광역시교육청', '경기도 여주시', '경기도 오산시',
       '대전광역시 중구', '부산광역시 해운대구', '강원도 삼척시', '인천광역시 미추홀구', '대전광역시 대덕구',
       '인천광역시 연수구', '충청남도 논산시', '인천광역시 남동구', '전라남도 광양시', '인천광역시 부평구',
       '경기도 화성시', '강원도 춘천시', '경기도 평택시', '경기도 연천군', '경상북도 군위군',
       '서울특별시 양천구', '충청북도 증평군', '전라남도 화순군', 

In [38]:
office_need[office_need['제공기관명'].str.contains('서울특별시')&office_need['개방장소명'].str.contains('센터')]

,개방장소명,소재지도로명주소,소재지지번주소,위도,경도,제공기관명
535,방학3동주민센터,서울특별시 도봉구 시루봉로 94,NaN,37.659141,127.027992,서울특별시 도봉구
536,방학3동주민센터,서울특별시 도봉구 시루봉로 94,NaN,37.659141,127.027992,서울특별시 도봉구
537,창1동주민센터,서울특별시 도봉구 해등로4길 3,NaN,37.648529,127.044045,서울특별시 도봉구
538,창2동주민센터,서울특별시 도봉구 덕릉로59길 73-3,NaN,37.641390,127.035640,서울특별시 도봉구
550,방학3동주민센터,서울특별시 도봉구 시루봉로 94,NaN,37.659141,127.027992,서울특별시 도봉구
...,...,...,...,...,...,...
6430,서대문구 남가좌1동 주민센터 지하2층 커뮤니티실2,서울특별시 서대문구 모래내로15길 37,서울특별시 서대문구 남가좌동 115-36,37.570880,126.916498,서울특별시 서대문구
6431,서대문구 남가좌1동 주민센터 지하2층 커뮤니티실3,서울특별시 서대문구 모래내로15길 37,서울특별시 서대문구 남가좌동 115-36,37.570880,126.916498,서울특별시 서대문구
6432,서대문구 남가좌1동 주민센터 3층 대강당,서울특별시 서대문구 모래내로15길 37,서울특별시 서대문구 남가좌동 115-36,37.570880,126.916498,서울특별시 서대문구
6433,서대문구 홍제3동 주민센터 2층 멀티학습실,서울특별시 서대문구 세검정로4길 32,서울특별시 서대문구 홍제동 7-13,37.593717,126.949739,서울특별시 서대문구


# 구별 위도경도

In [17]:
gu_loc.head()

,순번,시군구코드,시군구명_한글,시군구명_영문,ESRI_PK,위도,경도
0,1,11320,도봉구,Dobong-gu,0,37.665861,127.031767
1,2,11380,은평구,Eunpyeong-gu,1,37.617612,126.922700
2,3,11230,동대문구,Dongdaemun-gu,2,37.583801,127.050700
3,4,11590,동작구,Dongjak-gu,3,37.496504,126.944307
4,5,11545,금천구,Geumcheon-gu,4,37.460097,126.900155


In [18]:
gu_need = gu_loc[['시군구명_한글','위도','경도']]
gu_need

,시군구명_한글,위도,경도
0,도봉구,37.665861,127.031767
1,은평구,37.617612,126.922700
2,동대문구,37.583801,127.050700
3,동작구,37.496504,126.944307
4,금천구,37.460097,126.900155
5,구로구,37.495486,126.858121
6,종로구,37.599100,126.986149
7,강북구,37.646995,127.014716
8,중랑구,37.595379,127.093967
9,강남구,37.495985,127.066409


# 이것저것 전처리(볼필요X, 나중에 성공하면 위에다 반영할것임)

## 공원

공원번호는 전국 기준인듯하고, 면적은 몇몇개는 re써서 뽑아야할듯. 개원일은 큰 의미있을지는 모르겠다.주요시설에선 valuecount랑 unique본다음에 특수점으로 넣을지 생각하고, 지역이 대부분있는듯하지만 없으면 주소에서 뽑아야할듯. 

In [100]:
park_need = park[['공원명','공원주소','X좌표(WGS84)','Y좌표(WGS84)', '면적']]
park_need.면적 = park_need.면적.astype(str)
park_need

C:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,공원명,공원주소,X좌표(WGS84),Y좌표(WGS84),면적
0,남산도시자연공원,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡
1,길동생태공원,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,80683㎡
2,서울대공원,경기도 과천시 대공원광장로 102,127.019847,37.426449,9132690m²
3,서울숲,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,480994㎡
4,월드컵공원,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,2284085㎡
...,...,...,...,...,...
127,용마도시자연공원(사가정공원),서울특별시 중랑구 면목7동 산50,127.095997,37.579762,109635㎡
128,문화비축기지,서울특별시 마포구 증산로 87,126.893245,37.571718,140022㎡
129,경춘선숲길,노원구 공릉2동 산 82-2,127.076482,37.627077,211392㎡
130,율현공원,서울특별시 강남구 율현동 밤고개로24길(율현동 56),127.115594,37.472332,157535㎡


In [101]:
park_need.head(40)

,공원명,공원주소,X좌표(WGS84),Y좌표(WGS84),면적
0,남산도시자연공원,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡
1,길동생태공원,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,80683㎡
2,서울대공원,경기도 과천시 대공원광장로 102,127.019847,37.426449,9132690m²
3,서울숲,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,480994㎡
4,월드컵공원,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,2284085㎡
5,광화문시민열린마당,서울특별시 종로구 세종로 76-2,126.978023,37.575045,8948.1㎡
6,진관근린공원(구파발폭포),서울특별시 은평구 진관동 393,126.919216,37.637024,휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 1...
7,중마루근린공원,서울특별시 영등포구 영등포동2가 222,126.911267,37.519832,6456㎥
8,응봉공원,서울특별시 성동구 금호동1가 산1-72,127.021765,37.557228,80309㎡ 배수지 시설용량 : 201.000㎥ 송수 펌프용량 : 직송급수계통 30...
9,매화근린공원,서울특별시 강서구 등촌동 630-14,126.861150,37.557583,1980.4㎡


In [102]:
for i in range(len(park_need.면적)):
#     print(re.findall('([1-9]*?\.*[1-9]+?)㎡+?m*?²*?', park_need.면적[i]))
    print(re.findall('([0-9]+?\.?[0-9]?)㎡|([0-9]+?\.?[0-9]?)m²|([0-9]+?\.?[0-9]?)㎥', park_need.면적[i]))
    
     

[('2896887', '', ''), ('2454140', '', ''), ('442747', '', '')]
[('80683', '', '')]
[('', '9132690', '')]
[('480994', '', '')]
[('2284085', '', '')]
[('8948.1', '', '')]
[('18', '', ''), ('255', '', ''), ('90', '', ''), ('102', '', ''), ('55', '', ''), ('03', '', ''), ('36', '', '')]
[('', '', '6456')]
[('80309', '', ''), ('', '', '000'), ('', '', '30000'), ('', '', '16400')]
[('1980.4', '', '')]
[('5197.7', '', '')]
[('11467', '', '')]
[('229539', '', '')]
[]
[('', '', '60')]
[('61544', '', ''), ('40792', '', ''), ('6600', '', ''), ('14152', '', '')]
[('424106', '', '')]
[('560552', '', '')]
[('', '', '26696.8')]
[('75900', '', '')]
[]
[('', '', '201779')]
[('258991', '', '')]
[('08', '', '')]
[('', '', '171294.1')]
[('15000', '', '')]
[('297926', '', '')]
[('411972', '', '')]
[('98470', '', '')]
[('2038741.3', '', '')]
[('603715.2', '', '')]
[('108120', '', '')]
[('', '', '29619.3')]
[('29843.8', '', '')]
[('126026', '', '')]
[('135002.7', '', '')]
[('384026', '', '')]
[('', '', '1800

공원 넓이를 표현하는 방식이 다 다르다
그냥 1000제곱미터(㎡)라고 한곳이 있는 반면, 1000세제곱미터(㎥)로 한곳도, 1000띄어쓰기제곱미터(㎡) 한곳, 1000미터와 제곱(m²) 

배수지용량은 왜나오는데..;